## Roulette game logic: player profile, bet definition, payout definition, single roll simulation

In [6]:
# player definition as struct
mutable struct GamerProfile
    cash::Int64
end

#draw simulation
function rollTheRoulette(rouletteStyle::String)::Int64
    if rouletteStyle == "EUROPEAN"
        return rand(0:36)
    elseif rouletteStyle == "AMERICAN"
        #-1 for 00 field
        return rand(-1:36)
    else
        return 0
    end
end

#results parsing to color and parity
function rollResultsParser(result::Int)::Dict
    red = [1, 3, 5, 7, 9, 12, 14, 16, 18, 19, 21, 23, 25, 27, 30, 32, 34, 36]
    black = [2, 4, 6, 8, 10, 11, 13, 15, 17, 20, 22, 24, 26, 28, 29, 31, 33, 35]
    odd = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35]
    even = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36]
    
    parsedDict = Dict()
    
    if result in red
        parsedDict["color"] = "RED"
    elseif result in black
        parsedDict["color"] = "BLACK"
    else
        parsedDict["color"] = "GREEN"
    end
    
    if result in odd
        parsedDict["parity"] = "ODD"
    elseif result in even
        parsedDict["parity"] = "EVEN"
    else
        parsedDict["parity"] = "ZERO"
    end
    
    return parsedDict 
end

#bet definitions
#internal bets
function StraightUp(number::Int64, betAmmount::Int64)
    if (gamer.cash - betAmmount>=0)
        push!(bets,Dict("class"=>"StraightUp","number" =>number, "betAmmount" =>betAmmount))
        gamer.cash = gamer.cash - betAmmount
    end
end

function Split(number1::Int64, number2::Int64, betAmmount::Int64)
    tmp_list = []
    tmp = transpose(reshape(collect(1:36),3,12))
    zeroArrayV = zeros(1,3)
    tmp = vcat(zeroArrayV,tmp)
    zeroArrayH = ["*" for x in 1:13]
    tmp = hcat(zeroArrayH,tmp)
    tmp = hcat(tmp,zeroArrayH)
    zeroArrayV = ["*" "*" "*" "*" "*" ]
    tmp = vcat(zeroArrayV,tmp)
    tmp = vcat(tmp,zeroArrayV)
    positions = findall(x->x==number1, tmp)[1]
    push!(tmp_list, tmp[positions[1]-1,positions[2]])
    push!(tmp_list,tmp[positions[1]+1,positions[2]])
    push!(tmp_list,tmp[positions[1],positions[2]+1])
    push!(tmp_list,tmp[positions[1],positions[2]-1])
    boolean = number2 in tmp_list
    
    if (boolean == true) & (gamer.cash - betAmmount>=0)
        push!(bets,Dict("class"=>"Split","betAmmount"=>betAmmount,
                "numbers"=>[number1,number2]))
        gamer.cash = gamer.cash - betAmmount
    end
end

function Street(row::Int64, betAmmount::Int64)
    tmp = transpose(reshape(collect(1:36),3,12))
    zeroArrayV = zeros(1,3)
    tmp = vcat(zeroArrayV,tmp)
    if (gamer.cash-betAmmount>=0)
        push!(bets,Dict("class"=>"Street","betAmmount"=>betAmmount,
                "numbers"=>[tmp[row,1],tmp[row,2],tmp[row,3]]))
        gamer.cash = gamer.cash - betAmmount
    end
end

function Corner(num1::Int64, num2::Int64, num3::Int64, num4::Int64, betAmmount::Int64)
    nums = sort([num1,num2,num3,num4])
    tmp_list = []
    tmp = transpose(reshape(collect(1:36),3,12))
    zeroArrayV = zeros(1,3)
    tmp = vcat(zeroArrayV,tmp)
    zeroArrayH = ["*" for x in 1:13]
    tmp = hcat(zeroArrayH,tmp)
    tmp = hcat(tmp,zeroArrayH)
    zeroArrayV = ["*" "*" "*" "*" "*" ]
    tmp = vcat(zeroArrayV,tmp)
    tmp = vcat(tmp,zeroArrayV)
    positions1 = findall(x->x==nums[1], tmp)[1]
    positions2 = findall(x->x==nums[2], tmp)[1]
    positions3 = findall(x->x==nums[3], tmp)[1]
    positions4 = findall(x->x==nums[4], tmp)[1]
    if (positions1[1] == positions2[1]) & (positions1[2]+1 == positions2[2])&
    (positions1[1]+1 == positions3[1]) &
    (positions1[2] == positions3[2])&
    (positions1[1]+1 == positions4[1])&
    (positions1[2]+1 == positions4[2]) & (gamer.cash-betAmmount>=0)
        push!(bets,Dict("class"=>"Corner","betAmmount"=>betAmmount,
                "numbers"=>[nums[1],nums[2],nums[3],nums[4]]))
        gamer.cash = gamer.cash - betAmmount
    end
end

function Five(betAmmount::Int64)
    #only for american roulette
    if (gamer.cash-betAmmount>=0)
        push!(bets, Dict("class"=>"Five","betAmmount"=>betAmmount,
            "numbers"=>[-1,0,1,2,3]))
        gamer.cash = gamer.cash - betAmmount
    end
end

function Six(row1::Int64, row2::Int64, betAmmount::Int64)
    if (row2-1==row1) & (row2<14) & (gamer.cash-betAmmount>=0)
        tmp = transpose(reshape(collect(1:36),3,12))
        zeroArrayV = zeros(1,3)
        tmp = vcat(zeroArrayV,tmp)
        push!(bets,Dict("class"=>"Six","betAmmount"=>betAmmount,
                "numbers" => [tmp[row1,1],tmp[row1,2],tmp[row1,3],tmp[row2,1],tmp[row2,2],tmp[row2,3]]))
        gamer.cash = gamer.cash - betAmmount
    end
end

#bet definitions
#external bets
function Dozen(dozenNumber::Int64, betAmmount::Int64)
    if (dozenNumber>0) & (dozenNumber<4) & (gamer.cash-betAmmount>=0)
        if dozenNumber==1
            foo = [x for x in 1:12]
        elseif dozenNumber==2
            foo = [x for x in 13:24]
        else 
            foo = [x for x in 25:36]
        end
        push!(bets,Dict("class"=>"Dozen","numbers"=>foo,"betAmmount"=>betAmmount))
        gamer.cash = gamer.cash - betAmmount
    end
end

function Column(ColumnNumber::Int64, betAmmount::Int64)
    if (ColumnNumber>0) & (ColumnNumber<4) & (gamer.cash-betAmmount>=0)
        push!(bets,Dict("class"=>"Column","numbers"=>transpose(reshape(collect(1:36),3,12))[:,ColumnNumber], "betAmmount"=>betAmmount))
        gamer.cash = gamer.cash - betAmmount
    end
end

function Color(Col::String, betAmmount::Int64)
    if (Col in ["RED","BLACK"]) & (gamer.cash-betAmmount>=0)
        push!(bets,Dict("class"=>"Color", "colors" => Col, "betAmmount"=>betAmmount))
        gamer.cash = gamer.cash - betAmmount
    end
end

function EvenOdd(Ev::String, betAmmount::Int64)
    if (Ev in ["EVEN","ODD"]) & (gamer.cash-betAmmount>=0)
        push!(bets,Dict("class"=>"EvenOdd", "parity" => Ev, "betAmmount"=>betAmmount))
        gamer.cash = gamer.cash - betAmmount
    end
end

function HighLow(HL::String, betAmmount::Int64)
    if HL == "HIGH"
        foo = [i for i in 19:36]
    elseif HL == "LOW"
        foo = [i for i in 1:18]
    end
    
    if (gamer.cash-betAmmount>=0)
        push!(bets,Dict("class"=>"HighLow", "numbers" => foo, "betAmmount"=>betAmmount))
        gamer.cash = gamer.cash - betAmmount
    end
end

#payoffs
function payoffs(winning_number)
    for (i,j) in enumerate(bets)
        cls = bets[i]["class"]

        if cls == "StraightUp"
            if bets[i]["number"]==winning_number
                gamer.cash = gamer.cash +  36 * bets[i]["betAmmount"]
            end
        end

        if cls == "Split"
            if winning_number in bets[i]["numbers"]
                gamer.cash = gamer.cash + 18 * bets[i]["betAmmount"]
            end
        end

        if cls == "Street"
            if winning_number in bets[i]["numbers"]
                gamer.cash = gamer.cash + 12 * bets[i]["betAmmount"]
            end
        end

        if cls == "Corner"
            if winning_number in bets[i]["numbers"]
                gamer.cash = gamer.cash + 9 * bets[i]["betAmmount"]
            end
        end

        if cls == "Five"
            if winning_number in bets[i]["numbers"]
                gamer.cash = gamer.cash + 7 * bets[i]["betAmmount"]
            end
        end

        if cls == "Six"
            if winning_number in bets[i]["numbers"]
                gamer.cash = gamer.cash + 6 * bets[i]["betAmmount"]
            end
        end

        if cls == "Dozen"
            if winning_number in bets[i]["numbers"]
                gamer.cash = gamer.cash + 3 * bets[i]["betAmmount"]
            end
        end

        if cls == "Column"
            if winning_number in bets[i]["numbers"]
                gamer.cash = gamer.cash + 3 * bets[i]["betAmmount"]
            end
        end

        if cls == "HighLow"
            if winning_number in bets[i]["numbers"]
                gamer.cash = gamer.cash + 2 * bets[i]["betAmmount"]
            end
        end

        if cls == "Color"
            if bets[i]["colors"] == rollResultsParser(winning_number)["color"]
                gamer.cash = gamer.cash + 2 * bets[i]["betAmmount"]
            end
        end

        if cls == "EvenOdd"
            if bets[i]["parity"] == rollResultsParser(winning_number)["parity"]
                gamer.cash = gamer.cash + 2 * bets[i]["betAmmount"]
            end
        end

    end 
end

payoffs (generic function with 1 method)

In [7]:
using DelimitedFiles

# Monkey strategy

In [3]:
MC_results_od_betting = []
gamer = GamerProfile(10000)
count = 1
bet = 10
bets = Dict{Any, Any}[]

for i in 1:100000
    gamer = GamerProfile(10000)
    results_od_betting = [] 

    global count = 1
    global bet = 10

    while (count<= 1000) & (gamer.cash>0) & (gamer.cash <=20000)

        cashBeforeBet = gamer.cash
        bets = Dict{Any, Any}[]
        
        global bet = 10
        Color("RED",bet)
        payoffs(rollTheRoulette("EUROPEAN"))

        push!(results_od_betting,gamer.cash)
        global count += 1 
    end
    push!(MC_results_od_betting,results_od_betting)
end

In [4]:
writedlm( "monkey.csv",  MC_results_od_betting, ',')

# Martingale strategy

In [5]:
MC_results_od_betting = []
gamer = GamerProfile(10000)
count = 1
bet = 10
bets = Dict{Any, Any}[]

for i in 1:100000
    gamer = GamerProfile(10000)
    results_od_betting = [] 

    global count = 1
    global bet = 10

    while (count<= 1000) & (gamer.cash>0) & (gamer.cash <=20000)

        cashBeforeBet = gamer.cash
        bets = Dict{Any, Any}[]

        if count==1
            global bet = 10
            Color("RED",bet)
            payoffs(rollTheRoulette("EUROPEAN"))
        else
            Color("RED",bet)
            payoffs(rollTheRoulette("EUROPEAN"))
        end

        if gamer.cash > cashBeforeBet
            global bet = 10
        else
            global bet = 2 * bet
        end

        if gamer.cash < bet
           bet = gamer.cash 
        end


        push!(results_od_betting,gamer.cash)
        global count += 1 
    end
    push!(MC_results_od_betting,results_od_betting)
end

In [6]:
writedlm( "martingale.csv",  MC_results_od_betting, ',')

# Insurance strategy

In [5]:
MC_results_od_betting = []
gamer = GamerProfile(10000)
count = 1
bet = 10
bets = Dict{Any, Any}[]

for i in 1:100000
    gamer = GamerProfile(10000)
    results_od_betting = [] 

    global count = 1
    global bet = 10

    while (count<= 1000) & (gamer.cash>0) & (gamer.cash <=20000)

        cashBeforeBet = gamer.cash
        bets = Dict{Any, Any}[]
        cashAfterBetting = 0
        
        if count==1
            global bet = 10
            Dozen(1,bet)
            HighLow("LOW",bet*2)
            cashAfterBetting = gamer.cash
            payoffs(rollTheRoulette("EUROPEAN"))
        else
            Dozen(1,bet)
            HighLow("LOW",bet*2)
            cashAfterBetting = gamer.cash
            payoffs(rollTheRoulette("EUROPEAN"))
        end

        if (gamer.cash > cashBeforeBet) | (cashAfterBetting<gamer.cash)
            global bet = 10
        else
            global bet = 2 * bet
        end

        if gamer.cash < bet
           bet = gamer.cash 
        end


        push!(results_od_betting,gamer.cash)
        global count += 1 
    end
    push!(MC_results_od_betting,results_od_betting)
end

In [6]:
writedlm( "insurance.csv",  MC_results_od_betting, ',')

# James Bond strategy

In [9]:
MC_results_od_betting = []
gamer = GamerProfile(10000)
count = 1
bets = Dict{Any, Any}[]

for i in 1:100000
    gamer = GamerProfile(10000)
    results_od_betting = [] 

    global count = 1

    while (count<= 1000) & (gamer.cash>0) & (gamer.cash <=20000)

        cashBeforeBet = gamer.cash
        bets = Dict{Any, Any}[]
        
        HighLow("HIGH",140)
        Six(6,7,50)
        StraightUp(0,10)
        
        payoffs(rollTheRoulette("EUROPEAN"))


        push!(results_od_betting,gamer.cash)
        global count += 1 
    end
    push!(MC_results_od_betting,results_od_betting)
end

In [10]:
writedlm( "james_bond.csv",  MC_results_od_betting, ',')

# D'Alambert strategy

In [11]:
MC_results_od_betting = []
gamer = GamerProfile(10000)
count = 1
bet = 10
bets = Dict{Any, Any}[]

for i in 1:100000
    gamer = GamerProfile(10000)
    results_od_betting = [] 

    global count = 1
    global bet = 10

    while (count<= 1000) & (gamer.cash>0) & (gamer.cash <=20000)

        cashBeforeBet = gamer.cash
        bets = Dict{Any, Any}[]
        
        if count==1
            global bet = 10
            Color("RED",bet)
            payoffs(rollTheRoulette("EUROPEAN"))
        else
            Color("RED",bet)
            payoffs(rollTheRoulette("EUROPEAN"))
        end

        
        if gamer.cash > cashBeforeBet
            if bet == 10
                global bet = 10
            else
                global bet = bet - 10
            end
        else
            global bet = bet + 10
        end

        if gamer.cash < bet
           bet = gamer.cash 
        end


        push!(results_od_betting,gamer.cash)
        global count += 1 
    end
    push!(MC_results_od_betting,results_od_betting)
end

In [12]:
writedlm( "dalambert.csv",  MC_results_od_betting, ',')

# Middle strategy

In [7]:
MC_results_od_betting = []
gamer = GamerProfile(10000)
count = 1
bet = 10
bets = Dict{Any, Any}[]

for i in 1:100000
    gamer = GamerProfile(10000)
    results_od_betting = [] 

    global count = 1
    global bet = 10

    while (count<= 1000) & (gamer.cash>0) & (gamer.cash <=20000)

        cashBeforeBet = gamer.cash
        bets = Dict{Any, Any}[]
        cashAfterBetting = 0
        
        if count==1
            global bet = 10
            Dozen(2,bet)
            Color("BLACK",bet*2)
            cashAfterBetting = gamer.cash
            payoffs(rollTheRoulette("EUROPEAN"))
        else
            Dozen(2,bet)
            Color("BLACK",bet*2)
            cashAfterBetting = gamer.cash
            payoffs(rollTheRoulette("EUROPEAN"))
        end

        if (gamer.cash > cashBeforeBet) | (cashAfterBetting<gamer.cash)
            global bet = 10
        else
            global bet = 2 * bet
        end

        if gamer.cash < bet
           bet = gamer.cash 
        end


        push!(results_od_betting,gamer.cash)
        global count += 1 
    end
    push!(MC_results_od_betting,results_od_betting)
end

In [8]:
writedlm( "middle.csv",  MC_results_od_betting, ',')

# 64% strategy

In [9]:
MC_results_od_betting = []
gamer = GamerProfile(10000)
count = 1
bet = 10
bets = Dict{Any, Any}[]

for i in 1:100000
    gamer = GamerProfile(10000)
    results_od_betting = [] 

    global count = 1
    global bet = 10

    while (count<= 1000) & (gamer.cash>0) & (gamer.cash <=20000)

        cashBeforeBet = gamer.cash
        bets = Dict{Any, Any}[]
        
        if count==1
            global bet = 10
            Dozen(1,bet)
            Dozen(2,bet)
            payoffs(rollTheRoulette("EUROPEAN"))
        else
            Dozen(1,bet)
            Dozen(2,bet)
            payoffs(rollTheRoulette("EUROPEAN"))
        end

        if (gamer.cash > cashBeforeBet)
            global bet = 10
        else
            global bet = 3 * bet
        end

        if gamer.cash < bet
           bet = gamer.cash 
        end


        push!(results_od_betting,gamer.cash)
        global count += 1 
    end
    push!(MC_results_od_betting,results_od_betting)
end

In [10]:
writedlm( "64_prc.csv",  MC_results_od_betting, ',')

# Oscar's Grind Strategy

In [8]:
MC_results_od_betting = []
gamer = GamerProfile(10000)
count = 1
bet = 10
bets = Dict{Any, Any}[]

for i in 1:100000
    gamer = GamerProfile(10000)
    results_od_betting = [] 

    global count = 1
    global bet = 10

    while (count<= 1000) & (gamer.cash>0) & (gamer.cash <=20000)
        
        global initCasheBeforeSession = gamer.cash
        
        while (gamer.cash <= initCasheBeforeSession) & (count<= 1000) & (gamer.cash>0) & (gamer.cash <=20000)
           
            cashBeforeBet = gamer.cash
            bets = Dict{Any, Any}[]
            
            if count==1
                global bet = 10
                Color("RED",bet)
                payoffs(rollTheRoulette("EUROPEAN"))
            else
                Color("RED",bet)
                payoffs(rollTheRoulette("EUROPEAN"))
            end            
            
            if gamer.cash > cashBeforeBet
                global bet = bet + 10
            else
               global bet = 10 
            end
            
            if gamer.cash < bet
               bet = gamer.cash 
            end
            global count += 1 
            push!(results_od_betting,gamer.cash)
        end

    end
    push!(MC_results_od_betting,results_od_betting)
end

In [9]:
writedlm( "oscar.csv",  MC_results_od_betting, ',')